In [2]:
from scipy.optimize import minimize
import numpy as np
from sklearn import linear_model
import matplotlib.pyplot as plt
import math
from scipy import linalg
from scipy.linalg import block_diag
from scipy.stats import levy_stable
import sklearn
from sklearn.datasets.samples_generator import make_blobs
from sklearn.metrics import average_precision_score
from subprocess import call

In [3]:
def fit(A, x):
    return A.dot(x)

def l1_der(params, A, b):
    b_hat = fit(A, params) - b
    return (A.T).dot( np.sign(b_hat) )

def L1(A):
    return np.sum(np.sum(abs(A)))

def l1_loss(params, A, b):
    return np.sum(np.abs(b - fit(A, params)))

def l1_regression(A, b):
    
    x0 = np.linalg.pinv(A).dot(b)
    output = minimize(l1_loss, x0, args=(A, b), jac = l1_der, method = 'CG')
    
    return output.x, l1_loss(output.x, A, b)

In [4]:
#Our algorithm

def uniform_median_k_col_selection(A, k, num_repeat = 10): 
    
    best_base_col = np.array([[]])
    best_loss = math.inf
    
    r = int(np.min([50, np.floor(np.shape(A)[1]/k)]))
    
    for itr in range(num_repeat):
        
        m = np.shape(A)[1]
        pos = np.random.permutation(m)
        base_col_before_median = A[ :, pos[:r * k] ] / np.sum( np.abs(A[ :, pos[:r * k] ]), axis = 0 )
        scale = np.sum( np.abs(A[ :, pos[:r * k] ]), axis = 0 )
        scale = np.median(scale)
        base_col = np.zeros([np.shape(A)[0], k])
        for i in range(k):
            base_col[: , i] = np.median(base_col_before_median[:, i * r : (i + 1) * r], axis = 1) * scale
        cur_losses = np.array([0] * m)
        
        for i in range(m):
            cur_param, cur_losses[i] = l1_regression(base_col, A[:, i])
        cur_loss = sum(cur_losses)
            
        if(cur_loss < best_loss):
            best_loss = cur_loss
            best_base_col = base_col
    
    
    n = np.shape(A)[0]
    d = np.shape(A)[1]
    b_rank = np.shape(best_base_col)[1]
    
    V = np.zeros([b_rank, d])
    final_loss = 0
    for i in range(d):
        V[:, i], cur_loss = l1_regression(best_base_col, A[:, i])
        final_loss = final_loss + cur_loss
        
    return best_base_col, V, final_loss

In [5]:
#ICML 2017

def uniform_k_col_selection(A, k, num_repeat = 10): 
    
    best_base_col = np.array([[]])
    best_loss = math.inf
    
    for itr in range(num_repeat):
        
        m = np.shape(A)[1]
        pos = np.random.permutation(m)
        base_col = A[ :, pos[:k] ]
        cur_losses = np.array([0] * m)
        
        for i in range(m):
                cur_param, cur_losses[i] = l1_regression(base_col, A[:, i])
        cur_loss = sum(cur_losses)
            
        if(cur_loss < best_loss):
            best_loss = cur_loss
            best_base_col = base_col
    
    
    n = np.shape(A)[0]
    d = np.shape(A)[1]
    b_rank = np.shape(best_base_col)[1]
    
    V = np.zeros([b_rank, d])
    final_loss = 0
    for i in range(d):
        V[:, i], cur_loss = l1_regression(best_base_col, A[:, i])
        final_loss = final_loss + cur_loss
        
    return best_base_col, V, final_loss
        

In [6]:
#SVD

def F_low_rank_approx(A, k): 
    U, s, Vh = linalg.svd(A, full_matrices=False)
    t = s
    t[k:] = 0
    S = np.diag(t)
    
    return U.dot(S).dot(Vh), sum(sum((U.dot(S).dot(Vh)-A)*(U.dot(S).dot(Vh)-A)))
    

In [7]:
#STOC 2017

def L1_low_rank_approx_one_time(A, k):
    
    t = 4 * k
    t1 = t
    t2 = 3 * t1
    
    n = np.shape(A)[0]
    d = np.shape(A)[1]
    
    S = np.random.standard_cauchy([t1, n])
    R = np.random.standard_cauchy([d, t1])
    
    if(t2 < n):
        T1 = np.random.standard_cauchy([t2, n])
    else:
        T1 = np.identity(n)
    if(t2 < d):
        T2 = np.random.standard_cauchy([d, t2])
    else:
        T2 = np.identity(n)
    
    L = T1.dot(A).dot(R)
    N = S.dot(A).dot(T2)
    M = T1.dot(A).dot(T2)
    
    L_inv = np.linalg.pinv(L)
    N_inv = np.linalg.pinv(N)
    
    Y = L.dot(L_inv).dot(M).dot(N_inv).dot(N)
    UY, sy, VY = linalg.svd(Y, full_matrices=False)
    ty = sy
    ty[k:] = 0
    
    SY = np.diag(ty)
    
    X_hat = L_inv.dot(UY).dot(SY).dot(VY).dot(N_inv)
    
    return A.dot(R).dot(X_hat).dot(S).dot(A), sum(sum(np.abs(A.dot(R).dot(X_hat).dot(S).dot(A) - A)))

def L1_low_rank_approx(A, k, num_repeat = 10):
    
    best_loss = math.inf
    best_B = None
    
    for _ in range(num_repeat):
        B, loss = L1_low_rank_approx_one_time(A, k)
        if(loss < best_loss):
            best_loss = loss
            best_B = B
            
    return best_B, best_loss

In [8]:
#synthetic data generation

def gen_synthetic_matrix(n=500, k = 1, noise_type = 'root_cauchy', p = 1.1):
    
    noise_matrix = np.zeros([n,n])
    
    if noise_type == 'root_cauchy':
        noise_matrix = np.random.standard_cauchy([n, n])
        noise_matrix = np.sign(noise_matrix) * (np.sign(noise_matrix) * noise_matrix) ** (1/p)
    if noise_type == 'p_stable':
        noise_matrix = levy_stable.rvs(alpha = p, beta = 0, size=[n, n])
    
    X = np.random.randint(10, size = [n, n])
    X, loss = F_low_rank_approx(X, k)
    
    noise_matrix = noise_matrix / n ** 2 / 20 * np.sum(np.abs(X))
    A = X + noise_matrix
    
    return A, np.sum(np.sum(np.abs(noise_matrix)))

In [9]:
#comparison on synthetic data

def cmp_synthetic(k = 1, noise_type = 'root_cauchy'):
    
    print('#########################')
    
    print('synthetic data:')
    print('k:', k)
    
    A, ground_truth = gen_synthetic_matrix(500, k, noise_type)
    A = A.T
    print('total_weight:', L1(A))
    print('noise_weight:', ground_truth)
    
    B, loss = L1_low_rank_approx(A, k)
    print('L1LRA:', L1(A-B)/ground_truth)
    
    B, loss = F_low_rank_approx(A,  k)
    print('SVD:', L1(A-B)/ground_truth)
    
    B, V, loss = uniform_k_col_selection(A,  k)
    print('Uniform:', L1(A-B.dot(V))/ground_truth)
    
    B, V, loss = uniform_median_k_col_selection(A,  k)
    print('Uniform_mid:', L1(A-B.dot(V))/ground_truth)
    
    print('#########################')

In [37]:
cmp_synthetic(1, 'p_stable')

#########################
synthetic data:
k: 1
total_weight: 1326270.5238218263
noise_weight: 330087.7024360581
L1LRA: 1.74804030698863
SVD: 3.841230621411003
Uniform: 1.3048929995770813
Uniform_mid: 1.0036166848891235
#########################


In [38]:
cmp_synthetic(2, 'p_stable')

#########################
synthetic data:
k: 2
total_weight: 1274686.673174219
noise_weight: 278239.463512971
L1LRA: 3.619868518398919
SVD: 4.474453104251552
Uniform: 1.3541520629740298
Uniform_mid: 1.0330693926276815
#########################


In [39]:
cmp_synthetic(3, 'p_stable')

#########################
synthetic data:
k: 3
total_weight: 1249735.741948416
noise_weight: 255377.32342034968
L1LRA: 3.908389185829606
SVD: 4.752358661789228
Uniform: 1.3439831954126735
Uniform_mid: 1.0712182683200808
#########################


In [40]:
cmp_synthetic(4, 'p_stable')

#########################
synthetic data:
k: 4
total_weight: 1263511.8362692897
noise_weight: 264833.0502131734
L1LRA: 4.82510039670025
SVD: 4.592949391660939
Uniform: 1.3410117007388733
Uniform_mid: 1.0976244145205398
#########################


In [41]:
cmp_synthetic(5, 'p_stable')

#########################
synthetic data:
k: 5
total_weight: 1278089.6810855565
noise_weight: 278053.72724834067
L1LRA: 4.620595657369216
SVD: 4.391175470361519
Uniform: 1.342304720693403
Uniform_mid: 1.120325486629521
#########################


In [42]:
cmp_synthetic(6, 'p_stable')

#########################
synthetic data:
k: 6
total_weight: 1230682.535149784
noise_weight: 231888.47976141164
L1LRA: 5.40908541003624
SVD: 1.3298036013661652
Uniform: 1.4774241635763594
Uniform_mid: 1.1594937992584644
#########################


In [43]:
cmp_synthetic(7, 'p_stable')

#########################
synthetic data:
k: 7
total_weight: 1294159.6625642069
noise_weight: 293492.8182131115
L1LRA: 5.310374030743933
SVD: 4.133778947297198
Uniform: 1.3050961618453267
Uniform_mid: 1.1515053371305963
#########################


In [44]:
cmp_synthetic(8, 'p_stable')

#########################
synthetic data:
k: 8
total_weight: 1259502.07707308
noise_weight: 262659.1513098885
L1LRA: 5.688658798940587
SVD: 2.7867726178913648
Uniform: 1.4526511662195667
Uniform_mid: 1.1954587641923515
#########################


In [58]:
cmp_synthetic(9, 'p_stable')

#########################
synthetic data:
k: 9
total_weight: 1307849.867976689
noise_weight: 311549.9643112146
L1LRA: 5.630149921618203
SVD: 1.2953713101489326
Uniform: 1.3560996681770061
Uniform_mid: 1.178915650594507
#########################


In [46]:
cmp_synthetic(10, 'p_stable')

#########################
synthetic data:
k: 10
total_weight: 1228510.5315230787
noise_weight: 233293.98992028472
L1LRA: 5.44662950167869
SVD: 1.3585843300745972
Uniform: 1.4620169346081267
Uniform_mid: 1.2589530491778225
#########################


In [47]:
cmp_synthetic(1)

#########################
synthetic data:
k: 1
total_weight: 1355795.7577272335
noise_weight: 361891.01236739627
L1LRA: 2.9261345147008315
SVD: 3.6196257760132706
Uniform: 1.271313579086477
Uniform_mid: 1.0037425574437533
#########################


In [48]:
cmp_synthetic(2)

#########################
synthetic data:
k: 2
total_weight: 1264842.0296996308
noise_weight: 269609.26047198777
L1LRA: 4.235771705526642
SVD: 4.609563563866098
Uniform: 1.320358524019193
Uniform_mid: 1.0309248993141376
#########################


In [49]:
cmp_synthetic(3)

#########################
synthetic data:
k: 3
total_weight: 1306280.316456138
noise_weight: 311734.75359035213
L1LRA: 4.119058687385244
SVD: 4.038198662889504
Uniform: 1.2877921534837
Uniform_mid: 1.060211832303757
#########################


In [50]:
cmp_synthetic(4)

#########################
synthetic data:
k: 4
total_weight: 1267996.3987346173
noise_weight: 270693.4192673339
L1LRA: 4.3226020680132775
SVD: 4.486657205131688
Uniform: 1.3474648216101386
Uniform_mid: 1.100463431014291
#########################


In [57]:
cmp_synthetic(5)

#########################
synthetic data:
k: 5
total_weight: 1313391.204341366
noise_weight: 317360.23505069886
L1LRA: 4.354952150771891
SVD: 3.882342453046067
Uniform: 1.3238369381413795
Uniform_mid: 1.1089910761928228
#########################


In [52]:
cmp_synthetic(6)

#########################
synthetic data:
k: 6
total_weight: 1285704.0285657072
noise_weight: 289176.9080277399
L1LRA: 5.347679909110973
SVD: 3.9123465457829023
Uniform: 1.3526560300015447
Uniform_mid: 1.123628300678134
#########################


In [53]:
cmp_synthetic(7)

#########################
synthetic data:
k: 7
total_weight: 1288816.2787952854
noise_weight: 295572.949080861
L1LRA: 5.962500393755434
SVD: 4.1230667993963035
Uniform: 1.334959821082386
Uniform_mid: 1.1544675406393663
#########################


In [54]:
cmp_synthetic(8)

#########################
synthetic data:
k: 8
total_weight: 1280769.475587722
noise_weight: 286378.94657955057
L1LRA: 4.155868347608198
SVD: 1.2902879807202403
Uniform: 1.4014186826383184
Uniform_mid: 1.1746457205818852
#########################


In [55]:
cmp_synthetic(9)

#########################
synthetic data:
k: 9
total_weight: 1266716.1209430383
noise_weight: 269174.0262039277
L1LRA: 5.5489088617799185
SVD: 1.3376591662046122
Uniform: 1.3937518980169492
Uniform_mid: 1.1936557120305615
#########################


In [56]:
cmp_synthetic(10)

#########################
synthetic data:
k: 10
total_weight: 1285613.787816494
noise_weight: 289324.39584950934
L1LRA: 5.192068551621082
SVD: 1.3513936265821218
Uniform: 1.3959133839129325
Uniform_mid: 1.208802243900433
#########################


In [10]:
#generate real data

def gen_real_matrix(data_set = 'mfeat', k = 1, noise_type = 'root_cauchy', p = 1.1):
    
    X = np.array(np.zeros([2, 2]))
    
    if(data_set == 'mfeat'):
        sets = ["fourier", "zernike", "factors", "morphological", "karhunen", "pixel"]
        features = ["target", "target", "data", "target", "target", "data"]
        X = []
        for i, s in enumerate(sets):
            data = sklearn.datasets.fetch_mldata("uci-20070111 mfeat-%s" % s)
            feat = data[features[i]]
            if feat.shape[0] == 2000:
                X.append(feat)
            else:
                X.append(feat.transpose())
    
        X = np.hstack(X)
        
    if(data_set == 'isolet'):
        with open("data/isolet/isolet.data", "r") as inf:
            data = []
            for line in inf:
                tokens = line.strip().split(",")
                row = [np.float64(x) for x in tokens]
                data.append(row)

            X = np.array(data)
            X = X[: , :-1]
    
    n = np.shape(X)[0]
    d = np.shape(X)[1]
    
    noise_matrix = np.zeros([n,d])
    
    if noise_type == 'root_cauchy':
        noise_matrix = np.random.standard_cauchy([n, d])
        noise_matrix = np.sign(noise_matrix) * (np.sign(noise_matrix) * noise_matrix) ** (1/p)
    if noise_type == 'p_stable':
        noise_matrix = levy_stable.rvs(alpha = p, beta = 0, size=[n, d])
     
    X, loss = F_low_rank_approx(X, k)
    
    noise_matrix = noise_matrix/n/d/20 * np.sum(np.abs(X))
    
    X = X + noise_matrix
    
    return X, np.sum(np.sum(np.abs(noise_matrix)))
    

In [11]:
#comparison on real data

def cmp_real(data_set = 'mfeat', k = 1, noise_type = 'root_cauchy'):
    
    print('#########################')
    
    print('data_set:', data_set)
    print('k:', k)
    
    A, ground_truth = gen_real_matrix(data_set, k, noise_type)
    A = A.T
    print('total_weight:', L1(A))
    print('noise_weight:', ground_truth)
    
    B, loss = L1_low_rank_approx(A, k)
    print('L1LRA:', L1(A-B)/ground_truth)
    
    B, loss = F_low_rank_approx(A,  k)
    print('SVD:', L1(A-B)/ground_truth)
    
    B, V, loss = uniform_k_col_selection(A,  k)
    print('Uniform:', L1(A-B.dot(V))/ground_truth)
    
    B, V, loss = uniform_median_k_col_selection(A,  k)
    print('Uniform_mid:', L1(A-B.dot(V))/ground_truth)
    
    print('#########################')

In [61]:
cmp_real('isolet', 1, 'p_stable')

#########################
data_set: isolet
k: 1
total_weight: 441816.8612776701
noise_weight: 106534.56629125083
L1LRA: 3.685301059464859
SVD: 3.9961580378801163
Uniform: 1.2366967997376745
Uniform_mid: 1.0040606431856225
#########################


In [62]:
cmp_real('isolet', 2, 'p_stable')

#########################
data_set: isolet
k: 2
total_weight: 499747.02581769257
noise_weight: 119510.43688935638
L1LRA: 4.333884804292818
SVD: 4.014957395171908
Uniform: 1.3921173423774194
Uniform_mid: 1.1841898779799727
#########################


In [63]:
cmp_real('isolet', 3, 'p_stable')

#########################
data_set: isolet
k: 3
total_weight: 523809.0748687542
noise_weight: 133468.79603194518
L1LRA: 5.059388123709513
SVD: 3.6527881651140053
Uniform: 1.530942610294924
Uniform_mid: 1.3929053052403477
#########################


In [64]:
cmp_real('isolet', 4, 'p_stable')

#########################
data_set: isolet
k: 4
total_weight: 514762.3786157203
noise_weight: 116894.99645887161
L1LRA: 6.247865275278577
SVD: 4.198306068210552
Uniform: 1.8805870630867534
Uniform_mid: 1.57942117286315
#########################


In [65]:
cmp_real('isolet', 5, 'p_stable')

#########################
data_set: isolet
k: 5
total_weight: 513011.37766770856
noise_weight: 110280.43551907226
L1LRA: 5.735011720305959
SVD: 4.492436270186742
Uniform: 1.8113641240090546
Uniform_mid: 1.6479196680901613
#########################


In [66]:
cmp_real('isolet', 6, 'p_stable')

#########################
data_set: isolet
k: 6
total_weight: 644557.1059953162
noise_weight: 237255.2167442436
L1LRA: 3.4183709313777904
SVD: 2.1043162314168313
Uniform: 1.4660031171428256
Uniform_mid: 1.3431225892760468
#########################


In [67]:
cmp_real('isolet', 7, 'p_stable')

#########################
data_set: isolet
k: 7
total_weight: 593229.6659906162
noise_weight: 181243.28598043058
L1LRA: 3.8934478577180096
SVD: 2.8077395270579872
Uniform: 1.5903703159322178
Uniform_mid: 1.4721793264144731
#########################


In [68]:
cmp_real('isolet', 8, 'p_stable')

#########################
data_set: isolet
k: 8
total_weight: 527715.2824554718
noise_weight: 112130.98637015096
L1LRA: 5.687053772419038
SVD: 4.512633327620394
Uniform: 2.1314534953261597
Uniform_mid: 1.8429523582209875
#########################


In [69]:
cmp_real('isolet', 9, 'p_stable')

#########################
data_set: isolet
k: 9
total_weight: 525670.8576689904
noise_weight: 106651.85478683282
L1LRA: 6.7309808881807776
SVD: 4.723408521719126
Uniform: 2.0488315615352533
Uniform_mid: 2.005052074899167
#########################


In [70]:
cmp_real('isolet', 10, 'p_stable')

#########################
data_set: isolet
k: 10
total_weight: 568059.4229174358
noise_weight: 146322.2461510751
L1LRA: 5.91640194390267
SVD: 2.316535829420365
Uniform: 1.729917543918737
Uniform_mid: 1.6753146345859355
#########################


In [71]:
cmp_real('mfeat', 1, 'p_stable')

#########################
data_set: mfeat
k: 1
total_weight: 228419599.47742328
noise_weight: 86109519.17439042
L1LRA: 3.420527758186634
SVD: 2.2294370230658003
Uniform: 1.100286753051564
Uniform_mid: 1.0034020768315
#########################


In [72]:
cmp_real('mfeat', 2, 'p_stable')

#########################
data_set: mfeat
k: 2
total_weight: 198932771.4223723
noise_weight: 46649185.128182895
L1LRA: 3.8891096784949615
SVD: 1.6373587423127038
Uniform: 1.421190630753553
Uniform_mid: 1.036989095017017
#########################


In [73]:
cmp_real('mfeat', 3, 'p_stable')

#########################
data_set: mfeat
k: 3
total_weight: 204814743.15354404
noise_weight: 52403796.09398662
L1LRA: 4.679622216403125
SVD: 3.6153464728785
Uniform: 1.4139856437078204
Uniform_mid: 1.1202606806108777
#########################


In [74]:
cmp_real('mfeat', 4, 'p_stable')

#########################
data_set: mfeat
k: 4
total_weight: 194215917.8049368
noise_weight: 41779042.21546513
L1LRA: 5.914624039083712
SVD: 1.9566228922369955
Uniform: 1.5449439673445335
Uniform_mid: 1.2334589882583413
#########################


In [75]:
cmp_real('mfeat', 5, 'p_stable')

#########################
data_set: mfeat
k: 5
total_weight: 194030085.87810612
noise_weight: 41556028.05536769
L1LRA: 4.498222642200847
SVD: 2.0534673765069806
Uniform: 1.5199435453266872
Uniform_mid: 1.3196373488232092
#########################


In [76]:
cmp_real('mfeat', 6, 'p_stable')

#########################
data_set: mfeat
k: 6
total_weight: 192404548.68710268
noise_weight: 39935270.121089086
L1LRA: 7.282508911809922
SVD: 3.8789284083723468
Uniform: 1.5298495670992147
Uniform_mid: 1.321629730278347
#########################


In [77]:
cmp_real('mfeat', 7, 'p_stable')

#########################
data_set: mfeat
k: 7
total_weight: 189123829.3849648
noise_weight: 36647234.96282659
L1LRA: 6.437007046038447
SVD: 2.0199553490419184
Uniform: 1.5684442765644104
Uniform_mid: 1.3706293936028968
#########################


In [78]:
cmp_real('mfeat', 8, 'p_stable')

#########################
data_set: mfeat
k: 8
total_weight: 190757368.80637136
noise_weight: 38232812.23770664
L1LRA: 6.401908441239916
SVD: 1.5052885535390805
Uniform: 1.5876812685396793
Uniform_mid: 1.3421305692525265
#########################


In [79]:
cmp_real('mfeat', 9, 'p_stable')

#########################
data_set: mfeat
k: 9
total_weight: 194634953.24786243
noise_weight: 42142546.04084165
L1LRA: 6.477731844153904
SVD: 1.795808127288464
Uniform: 1.5166727295326814
Uniform_mid: 1.447917154498945
#########################


In [13]:
cmp_real('mfeat', 10, 'p_stable')

#########################
data_set: mfeat
k: 10
total_weight: 190586193.43540248
noise_weight: 38098676.430971965
L1LRA: 6.6627158155763615
SVD: 2.016739941018519
Uniform: 1.5356438938861163
Uniform_mid: 1.4177543033976563
#########################


In [81]:
cmp_real('isolet', 1)

#########################
data_set: isolet
k: 1
total_weight: 432749.51687879814
noise_weight: 98169.25840354376
L1LRA: 3.3348134623100067
SVD: 4.357313376273588
Uniform: 1.3081078963563988
Uniform_mid: 1.0056391576178263
#########################


In [82]:
cmp_real('isolet', 2)

#########################
data_set: isolet
k: 2
total_weight: 495001.33362534654
noise_weight: 115890.68370294853
L1LRA: 3.882716593751122
SVD: 4.201539603359612
Uniform: 1.3800834921426801
Uniform_mid: 1.107818607170326
#########################


In [83]:
cmp_real('isolet', 3)

#########################
data_set: isolet
k: 3
total_weight: 506361.5163661841
noise_weight: 117261.4496519992
L1LRA: 4.798442384231741
SVD: 4.171160197361597
Uniform: 1.8187090233929668
Uniform_mid: 1.436916797339518
#########################


In [84]:
cmp_real('isolet', 4)

#########################
data_set: isolet
k: 4
total_weight: 510351.28941185086
noise_weight: 113713.06492331893
L1LRA: 5.0848949929493665
SVD: 4.348368168076396
Uniform: 1.7820485770272156
Uniform_mid: 1.686346525897168
#########################


In [85]:
cmp_real('isolet', 5)

#########################
data_set: isolet
k: 5
total_weight: 542604.6705842974
noise_weight: 140884.615110062
L1LRA: 6.676501960926068
SVD: 3.5822430823711673
Uniform: 1.76228411927061
Uniform_mid: 1.5235266325335555
#########################


In [86]:
cmp_real('isolet', 6)

#########################
data_set: isolet
k: 6
total_weight: 520879.54405064846
noise_weight: 114528.2170650427
L1LRA: 5.388479315310451
SVD: 2.562790091224707
Uniform: 1.9032488519380895
Uniform_mid: 1.7448977825794818
#########################


In [87]:
cmp_real('isolet', 7)

#########################
data_set: isolet
k: 7
total_weight: 566348.615452439
noise_weight: 155483.3581463148
L1LRA: 5.013710802785392
SVD: 3.2586386007718957
Uniform: 1.5823470620565756
Uniform_mid: 1.5774207944600427
#########################


In [88]:
cmp_real('isolet', 8)

#########################
data_set: isolet
k: 8
total_weight: 529369.5497295886
noise_weight: 114929.03465675763
L1LRA: 7.329220523387763
SVD: 4.4770825698506895
Uniform: 1.9993947249931008
Uniform_mid: 1.7740082715207588
#########################


In [89]:
cmp_real('isolet', 9)

#########################
data_set: isolet
k: 9
total_weight: 586805.5277349411
noise_weight: 168858.95542829714
L1LRA: 6.299114765270358
SVD: 3.074481015739962
Uniform: 1.6655193929407885
Uniform_mid: 1.5780204907176882
#########################


In [90]:
cmp_real('isolet', 10)

#########################
data_set: isolet
k: 10
total_weight: 536536.183106136
noise_weight: 116063.4592834736
L1LRA: 7.086016914728179
SVD: 4.419122032227767
Uniform: 1.9983127605559314
Uniform_mid: 1.9165417214141038
#########################


In [91]:
cmp_real('mfeat', 1)

#########################
data_set: mfeat
k: 1
total_weight: 183925263.45739934
noise_weight: 41760068.78852912
L1LRA: 2.564800572957813
SVD: 4.361602847588104
Uniform: 1.298761486156381
Uniform_mid: 1.0060318178036594
#########################


In [92]:
cmp_real('mfeat', 2)

#########################
data_set: mfeat
k: 2
total_weight: 193667971.24707153
noise_weight: 41513040.99905234
L1LRA: 3.4637509600549534
SVD: 4.1608951542347405
Uniform: 1.5889846773150622
Uniform_mid: 1.0567514484409162
#########################


In [93]:
cmp_real('mfeat', 3)

#########################
data_set: mfeat
k: 3
total_weight: 196059916.87525734
noise_weight: 43939843.62434549
L1LRA: 3.3865667403632402
SVD: 4.3820584645064065
Uniform: 1.517379688049349
Uniform_mid: 1.20335635933768
#########################


In [94]:
cmp_real('mfeat', 4)

#########################
data_set: mfeat
k: 4
total_weight: 197449895.01806962
noise_weight: 45178576.63164523
L1LRA: 4.829598774877215
SVD: 4.191161749121343
Uniform: 1.4567568841666743
Uniform_mid: 1.2595411454696641
#########################


In [95]:
cmp_real('mfeat', 5)

#########################
data_set: mfeat
k: 5
total_weight: 193904981.69955555
noise_weight: 41635562.30536395
L1LRA: 5.282306863425414
SVD: 4.595069871510404
Uniform: 1.5207301055750715
Uniform_mid: 1.3003669605561192
#########################


In [96]:
cmp_real('mfeat', 6)

#########################
data_set: mfeat
k: 6
total_weight: 205547746.37824884
noise_weight: 53248277.7332978
L1LRA: 5.920126412597831
SVD: 3.5081951665698403
Uniform: 1.373710866113035
Uniform_mid: 1.2709413302343973
#########################


In [97]:
cmp_real('mfeat', 7)

#########################
data_set: mfeat
k: 7
total_weight: 192603999.13022503
noise_weight: 40324723.51023097
L1LRA: 6.144636341566159
SVD: 1.9040215349804879
Uniform: 1.596790225855327
Uniform_mid: 1.3557411400400254
#########################


In [98]:
cmp_real('mfeat', 8)

#########################
data_set: mfeat
k: 8
total_weight: 236858315.97807142
noise_weight: 84583364.67717734
L1LRA: 4.179974295475606
SVD: 2.2352290420778127
Uniform: 1.2610505413655857
Uniform_mid: 1.2033363715368
#########################


In [14]:
cmp_real('mfeat', 9)

#########################
data_set: mfeat
k: 9
total_weight: 198197265.6792286
noise_weight: 45829653.574734814
L1LRA: 6.340799867219061
SVD: 1.736225801211158
Uniform: 1.4813680539252247
Uniform_mid: 1.3795396343734414
#########################


In [100]:
cmp_real('mfeat', 10)

#########################
data_set: mfeat
k: 10
total_weight: 194980130.01020378
noise_weight: 42644824.39424349
L1LRA: 7.759728377996958
SVD: 1.8530168664090527
Uniform: 1.493717754265382
Uniform_mid: 1.3966170877034672
#########################
